## Preprocessing

In [1]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### Optimize step1 trying to drop column 'Status'

In [5]:
# Check status value counts and see if the column can be eliminated
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [6]:
# Drop the column as just 5 organizations from the entire dataset have inactive status - not to be added as feature column
application_df.drop(['STATUS'], axis=1, inplace=True)

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1500].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
df2 = pd.get_dummies(application_df)

In [13]:
# Review first five rows
print(df2)

        ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0          5000              1                       0                     1   
1        108590              1                       0                     0   
2          5000              0                       0                     0   
3          6692              1                       0                     0   
4        142590              1                       0                     0   
...         ...            ...                     ...                   ...   
34294      5000              0                       0                     0   
34295      5000              0                       0                     0   
34296      5000              0                       0                     0   
34297      5000              1                       0                     0   
34298  36500179              0                       0                     0   

       APPLICATION_TYPE_T19  APPLICATIO

In [14]:
# Split our preprocessed data into our features and target arrays
y = df2['IS_SUCCESSFUL']
X = df2.drop(columns='IS_SUCCESSFUL', axis=1)

### Optimize step2 - specify stratify=y and test_size to 50%

In [28]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.40, stratify=y)

In [29]:
# Check size of each of the datasets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(20579, 42) (13720, 42) (20579,) (13720,)


In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model (Optimize step3, trying a third hidden layer and increasing neurons 2-3 times as features)

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Get number of features
fcount = len(X_train_scaled[0])

# First hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation="relu", input_dim=fcount))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 70)                3010      
                                                                 
 dense_9 (Dense)             (None, 50)                3550      
                                                                 
 dense_10 (Dense)            (None, 30)                1530      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,121
Trainable params: 8,121
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Optimize step4 - increase epochs and batch_size

In [33]:
# Create callback that save the model's weight every 5 epochs
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', save_weights_only=True, save_freq=20)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=300, batch_size=16, callbacks=[mc])

Epoch 1/300
1287/1287 [==============================] - 4s 2ms/step - loss: 0.5721 - accuracy: 0.7198
Epoch 2/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5588 - accuracy: 0.7276
Epoch 3/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5562 - accuracy: 0.7271
Epoch 4/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5534 - accuracy: 0.7305
Epoch 5/300
1287/1287 [==============================] - 5s 4ms/step - loss: 0.5527 - accuracy: 0.7290
Epoch 6/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5514 - accuracy: 0.7310
Epoch 7/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5503 - accuracy: 0.7311
Epoch 8/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5492 - accuracy: 0.7311
Epoch 9/300
1287/1287 [==============================] - 3s 2ms/step - loss: 0.5491 - accuracy: 0.7318
Epoch 10/300
1287/1287 [==============================] - 3s 2ms/step - l

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

429/429 - 1s - loss: 0.5732 - accuracy: 0.7316 - 512ms/epoch - 1ms/step
Loss: 0.5732089281082153, Accuracy: 0.731632649898529


In [35]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")